In [1]:
from tools.yields import latex_line, pretty_yield_line, signal_latex_line
from tools.analysis import measurement
from tools.sample import Sample
from tools.plotting import yields_chart, sample_pie, significance_chart, syst_yields_chart
from tools import categories
from rootpy.plotting import set_style
from rootpy.plotting.base import convert_color
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from tabulate import tabulate
import ROOT
%matplotlib inline

Welcome to JupyROOT 6.10/08


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py:253: RuntimeWarning: creating converter for unknown type "_Atomic(bool)"
  value = getattr(object, key)


In [2]:
ll_file_default = '/Users/quentin/Desktop/htautau_draft1_wsinputs/leplep.root'
lh_file_default = '/Users/quentin/Desktop/htautau_draft1_wsinputs/lephad.root'
hh_file_default = '/Users/quentin/Desktop/htautau_draft1_wsinputs/hadhad.root'

ll_file_mjj = '/Users/quentin/Desktop/htautau_draft1_wsinputs/leplep.mjj.root'
lh_file_mjj = '/Users/quentin/Desktop/htautau_draft1_wsinputs/lephad.mjj.root'
hh_file_mjj = '/Users/quentin/atlas-hleptons/analyses/SM_Run2_Rel20.7/paper_histograms/workspaceInputs/hh_180502_v18_comb_Mjj_rebinned_100GeV_WithModJERVar_WithTheoryEnv.root'

ll_file_mmc_10 = '/Users/quentin/atlas-hleptons/analyses/SM_Run2_Rel20.7/paper_histograms/workspaceInputs/leplep_18-05-03-binnedCR.mmc_mlm.rebinALGO_WithTheoryEnv_WithModJERVar_fakes_zttTheoryConstrained_MGvsShShape.root'
#lh_file_mmc_10 = '/Users/quentin/atlas-hleptons/analyses/SM_Run2_Rel20.7/paper_histograms/workspaceInputs/rebinned_10GV_Size_lephad_fs56_MMC_Fine_With_MadVSShe_With_PSInput_Range_50200_WithFinalPSUnc_WithModJERVar_WithTheoryEnv_zttTheoryConstrained.root'
#lh_file_mmc_10 = '/Users/quentin/rebinned_10_GeV_Size_05252018_lephad_fs64_MMC_P_91a_Range_50200_WithMadVSSheAndPSInp_WithoutCrazyPDFBins_WithFinalPSUnc_WithModJERVar_WithTheoryEnv_zttTheoryConstrained.root'
lh_file_mmc_10 = '/Users/quentin/atlas-hleptons/analyses/SM_Run2_Rel20.7/paper_histograms/workspaceInputs/lep_had_18-05-29_MMC_with_TopCR_WithModJERVar_WithTheoryEnv_zttTheoryConstrained.root'
hh_file_mmc_10 = '/Users/quentin/atlas-hleptons/analyses/SM_Run2_Rel20.7/paper_histograms/workspaceInputs/hh_180502_v18_comb_MMC_rebinned_10GeV_WithModJERVar_WithTheoryEnv.root'

In [3]:
from rootpy.io import root_open

hh_rfile_default = root_open(hh_file_default, 'read')
lh_rfile_default = root_open(lh_file_default, 'read')
ll_rfile_default = root_open(ll_file_default, 'read')

hh_rfile_mjj     = root_open(hh_file_mjj, 'read')
lh_rfile_mjj     = root_open(lh_file_mjj, 'read')
ll_rfile_mjj     = root_open(ll_file_mjj, 'read')

hh_rfile_mmc_10 = root_open(hh_file_mmc_10, 'read')
lh_rfile_mmc_10 = root_open(lh_file_mmc_10, 'read')
ll_rfile_mmc_10 = root_open(ll_file_mmc_10, 'read')




In [4]:
def tabulate_2_inputs(samples, cats, rfile_def, rfile_new):
    table = []
    for sample in samples:
        for cat in cats:
            print sample, cat.name
            s = Sample(sample, 'red', 'dummy')
            nom_def = s.yields(rfile_def, cat.cats)[0]
            nom_new = s.yields(rfile_new, cat.cats)[0]
            table.append([sample, cat.name, 'nominal', nom_def, nom_new, nom_new / nom_def if nom_def !=0 else 0])
            systematics = s.systematics(cat.cats, rfile_def)
            if sample == 'Data':
                continue
            for sys in systematics[0]:
                sys_def = s.yields(rfile_def, cat.cats, sys)[0]
                sys_new = s.yields(rfile_new, cat.cats, sys)[0]
                table.append([sample, cat.name, sys, sys_def, sys_new, sys_new / sys_def if sys_def !=0 else 0])

    print tabulate(table, headers=['sample', 'category', 'hist', 'default', 'new', 'new / default'])

In [5]:
tabulate_2_inputs(['Data','Ztt', 'Fake'], categories['hadhad'], hh_rfile_default, hh_rfile_mmc_10)

Data hh vbf lowdr
Data hh vbf tight
Data hh vbf loose
Data hh boost tight
Data hh boost loose
Ztt hh vbf lowdr
Ztt hh vbf tight
Ztt hh vbf loose
Ztt hh boost tight
Ztt hh boost loose
Fake hh vbf lowdr
Fake hh vbf tight
Fake hh vbf loose
Fake hh boost tight
Fake hh boost loose
sample    category        hist                                                default        new    new / default
--------  --------------  ------------------------------------------------  ---------  ---------  ---------------
Data      hh vbf lowdr    nominal                                            179        179              1
Data      hh vbf tight    nominal                                            220        220              1
Data      hh vbf loose    nominal                                            121        121              1
Data      hh boost tight  nominal                                           4103       4103              1
Data      hh boost loose  nominal                                  

In [6]:
tabulate_2_inputs(['Data','Ztt', 'Fake'], categories['lephad_sr'], lh_rfile_default, lh_rfile_mmc_10)

Data lh vbf tight
Data lh vbf loose
Data lh boost tight
Data lh boost loose
Ztt lh vbf tight
Ztt lh vbf loose
Ztt lh boost tight
Ztt lh boost loose
Fake lh vbf tight
Fake lh vbf loose
Fake lh boost tight
Fake lh boost loose
sample    category        hist                                                default        new    new / default
--------  --------------  ------------------------------------------------  ---------  ---------  ---------------
Data      lh vbf tight    nominal                                            496        496              1
Data      lh vbf loose    nominal                                            318        318              1
Data      lh boost tight  nominal                                           6347       6347              1
Data      lh boost loose  nominal                                           6556       6556              1
Ztt       lh vbf tight    nominal                                            276.149    276.149          1
Ztt       lh 

In [7]:
tabulate_2_inputs(['Data','Ztt', 'Fake'], categories['leplep_sr'], ll_rfile_default, ll_rfile_mmc_10)

Data ll vbf tight
Data ll vbf loose
Data ll boost tight
Data ll boost loose
Ztt ll vbf tight
Ztt ll vbf loose
Ztt ll boost tight
Ztt ll boost loose
Fake ll vbf tight
Fake ll vbf loose
Fake ll boost tight
Fake ll boost loose
sample    category        hist                                                default        new    new / default
--------  --------------  ------------------------------------------------  ---------  ---------  ---------------
Data      ll vbf tight    nominal                                            188        188              1
Data      ll vbf loose    nominal                                            237        237              1
Data      ll boost tight  nominal                                           3444       3444              1
Data      ll boost loose  nominal                                           4124       4124              1
Ztt       ll vbf tight    nominal                                             97.7452    97.7452         1
Ztt       ll 